# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator
import random

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.3006989509363795),
 (1, 1.3006989509363795),
 (2, 1.3006989509363795),
 (3, 1.3006989509363795),
 (4, 1.3006989509363795)]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('is', 18), ('it', 18), ('what', 10)]),
 (1, [('a', 2), ('banana', 2)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.024215997968731084),
   (None, 0.03658672752452907),
   (None, 0.046216260771997475),
   (None, 0.0759764407627741),
   (None, 0.14663117985319607),
   (None, 0.18345639701873995),
   (None, 0.19862750060290912),
   (None, 0.22284070284321145),
   (None, 0.282577679218056),
   (None, 0.3489643859538414),
   (None, 0.35220724935729864),
   (None, 0.35466338997643443),
   (None, 0.37919535784386404),
   (None, 0.4011766216980369),
   (None, 0.43520675349498694),
   (None, 0.47780379677154783),
   (None, 0.48454279173105474)]),
 (1,
  [(None, 0.5341386920452825),
   (None, 0.5450666685656563),
   (None, 0.5570759959988305),
   (None, 0.5612779554217996),
   (None, 0.6379119269429593),
   (None, 0.6925894573198447),
   (None, 0.6984660872215095),
   (None, 0.7353951380944324),
   (None, 0.8428299546336447),
   (None, 0.9383341760899746),
   (None, 0.9460380100760039),
   (None, 0.9486324928590991),
   (None, 0.9930986108753707)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
import random

def MAP(num_list):
    return max(num_list)


def REDUCE(num_list):
    return max(num_list)


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]


record = RECORDREADER(100)
print(record)

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional)

print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[70, 51, 17, 79, 62, 81, 56, 84, 48, 27, 98, 48, 19, 71, 68, 39, 21, 83, 15, 49, 10, 49, 93, 7, 64, 38, 18, 16, 19, 53, 42, 35, 18, 75, 92, 95, 86, 31, 83, 66, 64, 10, 88, 91, 85, 72, 41, 41, 72, 71, 59, 45, 84, 14, 65, 100, 50, 76, 67, 85, 78, 66, 81, 36, 41, 23, 19, 84, 14, 49, 53, 54, 81, 23, 10, 33, 97, 83, 53, 82, 2, 67, 61, 90, 62, 42, 8, 83, 45, 7, 31, 31, 64, 19, 14, 50, 21, 87, 91, 15]
[[70, 51, 17, 79, 62], [81, 56, 84, 48, 27], [98, 48, 19, 71, 68], [39, 21, 83, 15, 49], [10, 49, 93, 7, 64], [38, 18, 16, 19, 53], [42, 35, 18, 75, 92], [95, 86, 31, 83, 66], [64, 10, 88, 91, 85], [72, 41, 41, 72, 71], [59, 45, 84, 14, 65], [100, 50, 76, 67, 85], [78, 66, 81, 36, 41], [23, 19, 84, 14, 49], [53, 54, 81, 23, 10], [33, 97, 83, 53, 82], [2, 67, 61, 90, 62], [42, 8, 83, 45, 7], [31, 31, 64, 19, 14], [50, 21, 87, 91, 15]]
100


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]


map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)

[(1, 88), (1, 96), (1, 6), (1, 57), (1, 90), (1, 5), (1, 77), (1, 38), (1, 31), (1, 18), (1, 99), (1, 69), (1, 86), (1, 37), (1, 40), (1, 51), (1, 72), (1, 54), (1, 23), (1, 5), (1, 72), (1, 73), (1, 47), (1, 98), (1, 12), (1, 0), (1, 2), (1, 94), (1, 64), (1, 64), (1, 35), (1, 91), (1, 78), (1, 10), (1, 88), (1, 2), (1, 77), (1, 39), (1, 91), (1, 81), (1, 1), (1, 58), (1, 53), (1, 32), (1, 69), (1, 93), (1, 29), (1, 44), (1, 9), (1, 16), (1, 56), (1, 24), (1, 22), (1, 12), (1, 98), (1, 83), (1, 36), (1, 20), (1, 97), (1, 76), (1, 83), (1, 63), (1, 85), (1, 48), (1, 13), (1, 63), (1, 1), (1, 11), (1, 63), (1, 60), (1, 92), (1, 92), (1, 97), (1, 33), (1, 15), (1, 94), (1, 8), (1, 30), (1, 73), (1, 71), (1, 90), (1, 97), (1, 16), (1, 26), (1, 18), (1, 69), (1, 5), (1, 42), (1, 82), (1, 39), (1, 64), (1, 25), (1, 29), (1, 10), (1, 70), (1, 94), (1, 53), (1, 16), (1, 57), (1, 84)]
[(1, [88, 96, 6, 57, 90, 5, 77, 38, 31, 18, 99, 69, 86, 37, 40, 51, 72, 54, 23, 5, 72, 73, 47, 98, 12, 0, 2, 9

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
def group_by_key(iterable):
    sorted_data = sorted(iterable, key=lambda x: x[0])
    grouped_data = {}
    for key, value in sorted_data:
        if key not in grouped_data:
            grouped_data[key] = [value]
        else:
            grouped_data[key].append(value)
    result = []
    for key, value in grouped_data.items():
        result.append((key, value))
    return result

def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)

[(1, 6), (1, 14), (1, 57), (1, 7), (1, 38), (1, 99), (1, 20), (1, 65), (1, 35), (1, 0), (1, 14), (1, 54), (1, 55), (1, 50), (1, 28), (1, 91), (1, 16), (1, 46), (1, 39), (1, 98), (1, 8), (1, 99), (1, 42), (1, 83), (1, 31), (1, 15), (1, 94), (1, 8), (1, 39), (1, 28), (1, 59), (1, 46), (1, 37), (1, 33), (1, 9), (1, 83), (1, 100), (1, 17), (1, 61), (1, 91), (1, 70), (1, 47), (1, 99), (1, 45), (1, 28), (1, 39), (1, 35), (1, 35), (1, 28), (1, 47), (1, 54), (1, 94), (1, 92), (1, 11), (1, 72), (1, 8), (1, 75), (1, 62), (1, 11), (1, 74), (1, 53), (1, 94), (1, 91), (1, 23), (1, 10), (1, 17), (1, 64), (1, 19), (1, 42), (1, 87), (1, 9), (1, 87), (1, 16), (1, 19), (1, 23), (1, 17), (1, 58), (1, 46), (1, 98), (1, 61), (1, 91), (1, 16), (1, 51), (1, 10), (1, 92), (1, 33), (1, 89), (1, 17), (1, 32), (1, 90), (1, 67), (1, 85), (1, 97), (1, 8), (1, 52), (1, 20), (1, 2), (1, 25), (1, 1), (1, 42)]
[(1, [6, 14, 57, 7, 38, 99, 20, 65, 35, 0, 14, 54, 55, 50, 28, 91, 16, 46, 39, 98, 8, 99, 42, 83, 31, 15, 94,

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [25]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs


# ===================================================================================================================================================================================

from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, word)

def REDUCE(key:str, value:Iterator[str]):
  yield key

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, ['', 'is', 'it', 'what']), (1, ['a', 'banana'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [26]:
from collections import defaultdict

def MAP(el_list):
    mapped_result = defaultdict(list)
    for t in el_list:
        if C(t):  # Предикат C
            mapped_result[t].append(t)
    return mapped_result.items()


def REDUCE(mapped_items):
    reduced_result = []
    print(mapped_items)
    for values in mapped_items:
        for value in values:
            reduced_result.append(value[0])
    return reduced_result

def C(t):
    return t[0] % 2 == 0  # Выбираем только кортежи с четным первым элементом


def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

record = RECORDREADER(100)
print(record)

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional)

print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[(52, 40, 29), (2, 0, 39), (53, 16, 9), (38, 34, 61), (53, 60, 31), (24, 11, 65), (73, 59, 25), (75, 2, 43), (43, 88, 32), (88, 30, 60), (65, 95, 67), (6, 35, 19), (50, 8, 20), (29, 17, 17), (67, 100, 94), (4, 14, 64), (17, 69, 95), (9, 94, 43), (97, 9, 2), (87, 23, 72), (82, 1, 56), (97, 16, 64), (41, 25, 59), (75, 32, 95), (58, 94, 57), (75, 68, 14), (1, 63, 41), (53, 22, 86), (68, 29, 52), (39, 98, 56), (92, 53, 76), (4, 12, 15), (79, 69, 80), (81, 40, 62), (32, 95, 36), (57, 82, 97), (38, 27, 14), (1, 71, 99), (28, 62, 45), (95, 93, 35), (64, 65, 26), (91, 89, 5), (44, 53, 49), (26, 80, 60), (6, 93, 88), (17, 48, 18), (15, 56, 71), (29, 41, 21), (69, 26, 62), (18, 1, 80), (24, 41, 31), (22, 34, 74), (93, 90, 1), (79, 11, 77), (91, 95, 58), (32, 64, 12), (84, 42, 95), (54, 70, 51), (59, 16, 20), (53, 1, 87), (4, 60, 25), (85, 92, 43), (62, 33, 53), (60, 25, 92), (7, 37, 61), (87, 59, 67), (95, 32, 40), (16, 89, 84), (46, 42, 16), (39, 56, 16), (95, 73, 25), (24, 51, 47), (62, 93, 34

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [27]:
S = set([1 ,12, 56])

def MAP(t):
  res_list = []
  for el in t:
    if el in S:
      res_list.append(el)
    res = tuple(res_list)
    return (res, res)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(66, 15, 70), (38, 88, 28), (47, 0, 72), (46, 12, 29), (14, 98, 27), (57, 7, 57), (45, 76, 26), (83, 32, 47), (5, 56, 34), (57, 85, 27), (59, 32, 88), (59, 87, 10), (32, 92, 24), (43, 2, 61), (32, 21, 66), (21, 0, 2), (41, 34, 29), (48, 89, 59), (35, 94, 4), (12, 36, 77), (68, 83, 88), (15, 41, 85), (45, 48, 50), (50, 26, 55), (47, 28, 96), (64, 52, 52), (4, 84, 33), (33, 72, 23), (57, 24, 45), (30, 87, 36), (70, 5, 23), (40, 62, 64), (63, 24, 10), (3, 28, 56), (57, 53, 23), (76, 94, 61), (23, 37, 63), (83, 94, 28), (0, 47, 11), (92, 64, 55), (8, 72, 31), (6, 90, 23), (73, 81, 72), (20, 13, 92), (89, 53, 39), (3, 36, 24), (62, 53, 43), (67, 84, 6), (91, 61, 59), (75, 72, 59), (75, 83, 31), (54, 55, 66), (66, 77, 97), (56, 3, 53), (89, 1, 33), (28, 8, 47), (74, 23, 34), (89, 55, 81), (67, 55, 55), (86, 49, 100), (42, 38, 7), (61, 9, 28), (28, 29, 38), (13, 91, 8), (78, 64, 50), (66, 99, 29), (64, 7, 100), (90, 8, 20), (16, 81, 9), (18, 94, 6), (17, 27, 40), (24, 97, 68), (29, 25, 47), 

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [28]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(46, 94, 38), (20, 80, 80), (44, 95, 37), (72, 77, 99), (75, 49, 23), (75, 59, 48), (80, 73, 55), (97, 35, 66), (66, 38, 6), (49, 99, 35), (20, 69, 46), (82, 18, 97), (64, 30, 78), (63, 12, 80), (75, 6, 19), (25, 84, 50), (18, 45, 72), (18, 42, 26), (35, 95, 80), (0, 98, 76), (25, 6, 44), (99, 97, 64), (70, 44, 14), (60, 53, 0), (48, 83, 95), (77, 89, 59), (34, 98, 12), (41, 52, 53), (8, 39, 43), (44, 63, 64), (6, 31, 61), (86, 46, 41), (25, 45, 35), (60, 39, 87), (7, 60, 75), (40, 9, 31), (93, 84, 71), (28, 35, 17), (20, 39, 43), (70, 19, 0), (33, 0, 96), (55, 33, 4), (48, 25, 18), (71, 100, 67), (40, 81, 95), (16, 2, 80), (33, 12, 65), (91, 63, 36), (57, 16, 5), (40, 62, 69), (81, 75, 11), (42, 46, 24), (99, 31, 87), (37, 37, 59), (94, 13, 48), (2, 92, 11), (27, 73, 89), (60, 95, 90), (15, 15, 48), (65, 23, 90), (91, 58, 58), (98, 69, 59), (64, 75, 24), (62, 88, 77), (39, 89, 99), (12, 64, 65), (30, 73, 44), (87, 100, 28), (99, 53, 52), (80, 60, 42), (38, 8, 10), (40, 21, 66), (78, 

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [29]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
  if len(values) == 2:
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)

[(1, 0), (3, 3), (0, 0), (3, 1), (1, 1), (3, 1), (3, 2), (0, 0), (3, 3), (2, 2), (0, 1), (3, 2), (3, 2), (3, 3), (2, 1), (3, 2), (3, 1), (2, 1), (1, 2), (0, 3), (3, 0), (3, 0), (0, 2), (3, 1), (3, 3), (0, 1), (0, 3), (0, 3), (1, 2), (0, 2), (2, 3), (3, 2), (0, 1), (3, 1), (0, 1), (1, 3), (0, 0), (2, 0), (1, 0), (3, 3), (2, 1), (0, 0), (0, 3), (0, 2), (2, 1), (1, 1), (1, 1), (0, 2), (1, 1), (2, 0), (0, 1), (3, 2), (0, 2), (0, 1), (2, 2), (2, 3), (2, 1), (1, 0), (3, 1), (3, 1), (1, 0), (1, 2), (3, 3), (0, 1), (2, 3), (1, 1), (1, 2), (0, 0), (3, 3), (2, 2), (1, 0), (0, 2), (1, 3), (2, 3), (3, 2), (1, 1), (2, 3), (1, 0), (3, 2), (1, 3), (1, 2), (1, 3), (0, 2), (2, 1), (2, 3), (2, 3), (1, 3), (2, 0), (1, 1), (1, 2), (1, 3), (0, 2), (2, 1), (1, 3), (0, 0), (3, 3), (1, 2), (1, 0), (1, 1), (1, 1)]
[((3, 1), (3, 1)), ((0, 3), (0, 3)), ((3, 3), (3, 3)), ((2, 0), (2, 0)), ((2, 0), (2, 0)), ((0, 1), (0, 1)), ((2, 2), (2, 2)), ((3, 0), (3, 0)), ((1, 3), (1, 3)), ((1, 1), (1, 1)), ((1, 3), (1, 3)), 

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [30]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple(count):
  data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
    return [get_random_tuple(3) for i in range(count)]

def MAP(t: Tuple):
  return (t.data, t.rel_id)

def REDUCE(key, values: Iterator[NamedTuple]):
  if values == [rels[0]]: # если соответствующее значение является списком  [R] , создайте пару  (t,t)
    return (key, key)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)

[<__main__.Tuple object at 0x000001C367FD3760>, <__main__.Tuple object at 0x000001C367FD3400>, <__main__.Tuple object at 0x000001C367FD3520>, <__main__.Tuple object at 0x000001C367FD3310>, <__main__.Tuple object at 0x000001C367FD3580>, <__main__.Tuple object at 0x000001C367FD3430>, <__main__.Tuple object at 0x000001C367FD3790>, <__main__.Tuple object at 0x000001C367FD3EE0>, <__main__.Tuple object at 0x000001C367FD30A0>, <__main__.Tuple object at 0x000001C367FD3DF0>, <__main__.Tuple object at 0x000001C367FD3640>, <__main__.Tuple object at 0x000001C358B94A00>, <__main__.Tuple object at 0x000001C358B94AC0>, <__main__.Tuple object at 0x000001C358B94BE0>, <__main__.Tuple object at 0x000001C358B943A0>, <__main__.Tuple object at 0x000001C358B94730>, <__main__.Tuple object at 0x000001C358B94640>, <__main__.Tuple object at 0x000001C358B94E80>, <__main__.Tuple object at 0x000001C358B94490>, <__main__.Tuple object at 0x000001C358B94220>, <__main__.Tuple object at 0x000001C358B94A30>, <__main__.Tu

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [31]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple():
  data = (random.randint(0, 3), random.randint(0, 3))
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  if t.rel_id == rels[0]:
    return (t.data[1], (t.rel_id, t.data[0]))
  else:
    return (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values: Iterator[NamedTuple]):
  res = []
  for v in values:
    res.append((v[0], key, v[1]))
  return res

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[<__main__.Tuple object at 0x000001C367FA7A60>, <__main__.Tuple object at 0x000001C367FA7C10>, <__main__.Tuple object at 0x000001C367FA7A90>, <__main__.Tuple object at 0x000001C367FB4F70>, <__main__.Tuple object at 0x000001C367FB4DC0>, <__main__.Tuple object at 0x000001C367FB4C40>, <__main__.Tuple object at 0x000001C367FB4AF0>, <__main__.Tuple object at 0x000001C367FB4430>, <__main__.Tuple object at 0x000001C367FB4250>, <__main__.Tuple object at 0x000001C367FB4CA0>, <__main__.Tuple object at 0x000001C367FB4AC0>, <__main__.Tuple object at 0x000001C367FD3880>, <__main__.Tuple object at 0x000001C367FD35E0>, <__main__.Tuple object at 0x000001C367FD38E0>, <__main__.Tuple object at 0x000001C367FD3850>, <__main__.Tuple object at 0x000001C367FD3B80>, <__main__.Tuple object at 0x000001C367FD3730>, <__main__.Tuple object at 0x000001C367FD3EB0>, <__main__.Tuple object at 0x000001C367FD3A90>, <__main__.Tuple object at 0x000001C367FD3FD0>, <__main__.Tuple object at 0x000001C367FD3B50>, <__main__.Tu

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [32]:
def get_random_tuple():
  return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  return (t[0], t[1])

def tetta(values):
  return sum(values)


def REDUCE(key, values: Iterator[NamedTuple]):
  x = tetta(values)
  return (key, x)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[(1, 0, 3), (1, 0, 3), (1, 1, 1), (0, 1, 2), (0, 2, 0), (1, 0, 0), (2, 2, 0), (2, 3, 0), (0, 0, 3), (0, 0, 2), (0, 2, 3), (1, 1, 0), (1, 1, 2), (2, 1, 3), (1, 0, 0), (2, 2, 2), (0, 0, 0), (0, 3, 0), (3, 3, 2), (2, 2, 1), (1, 3, 0), (0, 2, 3), (1, 0, 1), (3, 3, 2), (1, 0, 3), (1, 3, 2), (3, 1, 3), (2, 3, 3), (3, 2, 0), (3, 1, 2), (0, 3, 1), (1, 1, 1), (3, 0, 2), (0, 2, 3), (3, 2, 2), (1, 2, 1), (1, 2, 1), (0, 2, 3), (2, 2, 1), (3, 2, 3), (1, 2, 2), (3, 1, 3), (2, 3, 2), (2, 3, 3), (1, 3, 3), (2, 0, 2), (1, 1, 3), (2, 3, 2), (2, 2, 1), (2, 3, 2), (1, 1, 1), (2, 2, 3), (3, 0, 0), (0, 0, 3), (2, 1, 3), (0, 2, 2), (2, 1, 0), (3, 1, 3), (1, 2, 3), (1, 3, 3), (1, 2, 0), (1, 1, 0), (0, 3, 0), (0, 0, 2), (3, 1, 3), (2, 1, 1), (3, 1, 3), (2, 1, 1), (3, 0, 2), (3, 3, 1), (1, 3, 0), (3, 3, 3), (0, 1, 1), (0, 2, 0), (3, 2, 2), (1, 3, 0), (2, 1, 3), (0, 1, 0), (3, 0, 1), (0, 0, 0), (3, 2, 0), (0, 3, 0), (3, 0, 1), (1, 3, 3), (1, 0, 1), (2, 3, 0), (2, 0, 0), (1, 1, 1), (3, 0, 3), (1, 3, 1), (0, 2, 2)

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [33]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [34]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j, k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [35]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [36]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [37]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      yield ((0, i, j), small_mat[i,j]) # первая матрица

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((1, j, k), big_mat[j,k]) # вторая матрица

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def MAP_MUL(k1, v1):
  (i, k) = k1
  yield ((i, k), v1)


def REDUCE_MUL(key, values):
  res_el_value = 0
  for v in values:
    res_el_value += v
  yield (key, res_el_value)


def GET_JOINED():
  for j in joined:
    yield j

joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [38]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# ===================================================================================================================================================================================

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица
  yield second_mat


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution)) # should return true


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.013998545059063125), ((0, 1), 0.006146411005643418), ((0, 2), 0.0013094070280139968), ((0, 3), 0.006761229708196888), ((0, 4), 0.001640529289576924), ((0, 5), 0.002799185187369619), ((0, 6), 0.0105191523833935), ((0, 7), 0.0013711064884254377), ((0, 8), 0.013411951239838366), ((0, 9), 0.005426143970943269), ((0, 10), 0.007609049702543101), ((0, 11), 0.006972608712276354), ((0, 12), 0.003454543044118363), ((0, 13), 0.013734923700417226), ((0, 14), 0.00011522547046654885), ((0, 15), 0.009908856673628218), ((0, 16), 0.012300277275695567), ((0, 17), 0.013799111322498551), ((0, 18), 0.013437119711956177), ((0, 19), 0.0017433486286859733), ((0, 20), 0.0010128292848794714), ((0, 21), 0.004086297228970294), ((0, 22), 0.0003102038136482358), ((0, 23), 0.005563291398424726), ((0, 24), 0.0005735526644171284), ((0, 25), 0.01207463764694144), ((0, 26), 0.011518737208141352), ((0, 27), 0.010662534328019476), ((0, 28), 0.00844885799778191

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [39]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# ===================================================================================================================================================================================

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица

  global maps
  split_size =  int(np.ceil(len(first_mat)/maps))
  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица

  split_size =  int(np.ceil(len(second_mat)/maps))
  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution)) # should return true


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.7786588102935185), ((0, 1), 0.04411830982713003), ((0, 2), 0.6250213647582906), ((0, 3), 0.2694024652246955), ((0, 4), 0.43171032548605665), ((0, 5), 0.3159699466628024), ((0, 6), 0.8139487383527247), ((0, 7), 0.11011351294897491), ((0, 8), 0.8072138207134778), ((0, 9), 0.06917088875281344), ((0, 10), 0.4757974860186921), ((0, 11), 0.810771084854452), ((0, 12), 0.6227332627552942), ((0, 13), 0.15855956580427538), ((0, 14), 0.4900719604479298), ((0, 15), 0.5280077873490829), ((0, 16), 0.687060519015845), ((0, 17), 0.11513308284726824), ((0, 18), 0.13499692605835636), ((0, 19), 0.04060358763171015), ((0, 20), 0.7432166301135735), ((0, 21), 0.49647298660745137), ((0, 22), 0.00796473286641057), ((0, 23), 0.5405431423683336), ((0, 24), 0.6038797312278291), ((0, 25), 0.3606552122192031), ((0, 26), 0.2617805715365355), ((0, 27), 0.35025526160360504), ((0, 28), 0.2200768788301147), ((0, 29), 0.3649829972891148), ((0, 30), 0.0058630

True